In [1]:
import os
from sklearn.datasets import fetch_covtype
import random
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import accuracy_score, f1_score
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import random
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import accuracy_score, f1_score
import os
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
X_train = pd.read_csv('X_train_nothing.csv', index_col = ['Unnamed: 0'])
y_train = pd.read_csv('y_train_nothing.csv', index_col = ['Unnamed: 0'])
X_test = pd.read_csv('X_test_nothing.csv', index_col = ['Unnamed: 0'])
y_test = pd.read_csv('y_test_nothing.csv', index_col = ['Unnamed: 0'])

In [4]:
cat_col_names = ['Килип1', 'Килип4', 'Тромболит3', 'ГБ6', 'АКШ', 'Килип3', 'Килип2', 'Кл.вар.ИМ1', 'Хр.оч.инф2']
num_col_names = ['Creatin', 'ФУ1', 'ИндНор1', 'Возраст', 'Pсрлао(Адср)1', 'ПИ', 'Рла1']

In [5]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, NodeConfig, TabNetModelConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig

In [9]:
data_config = DataConfig(
    target=['target'],
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform = "quantile_normal"
)

trainer_config = TrainerConfig(
    auto_lr_find = False,
    batch_size = 16,
    max_epochs = 10,
    gpus = 0,
    gradient_clip_val = 10,
    load_best = True
)

model_config = NodeConfig(
    task = "classification",
    num_layers = 2,
    num_trees = 1024,
    learning_rate = 0.1,
    embed_categorical = False,
    metrics = ["accuracy"],
)

tabular_model = TabularModel(
    data_config = data_config,
    model_config = model_config,
    optimizer_config = OptimizerConfig(),
    trainer_config = trainer_config,
)

In [10]:
X_train['target'] = y_train
X_test['targetд'] = y_test

In [11]:
tabular_model.fit(train = X_train, test = X_test)

C:\Users\olyav\AppData\Roaming\Python\Python38\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\olyav\AppData\Roaming\Python\Python38\site-packages\sklearn\preprocessing\_data.py:2367: UserWarning: n_quantiles (1000) is greater than the total number of samples (640). n_quantiles is set to n_samples.
  warnings.warn("n_quantiles (%s) is greater than the total number "
C:\Users\olyav\AppData\Roaming\Python\Python38\site-packages\pytorch_tabular\models\node\odst.py:152: UserWarning: Data-aware initialization is performed on less than 1000 data points. This may cause instability.To avoid potential problems, run this model on a data batch with at least 1000 data samples.You can do so manually before training. Use with torch.no_grad() for memory efficiency.
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\pytorch_lightning\uti

In [12]:
pred_df = tabular_model.predict(X_test)
pred_df.head()

,Килип1,ИндНор1,Creatin,Тромболит3,Килип4,ФУ1,ГБ6,АКШ,Возраст,Килип3,Рла1,Килип2,Кл.вар.ИМ1,Хр.оч.инф2,Pсрлао(Адср)1,ПИ,target,0_probability,1_probability,prediction
268,0.0,2.290910,-0.003685,0.0,0.0,-1.030706,0.0,0.0,0.676472,1.0,-0.785664,0.0,0.0,0.0,-2.429174,-0.289795,0,0.647570,0.352430,0
555,0.0,-0.027838,-0.513335,0.0,0.0,1.859569,0.0,0.0,-0.403983,0.0,-0.239671,1.0,0.0,0.0,0.655215,0.040012,0,0.995992,0.004008,0
568,1.0,-0.987511,-1.249497,0.0,0.0,1.591951,0.0,0.0,-0.635509,0.0,-0.938982,0.0,1.0,1.0,-0.578541,0.040012,0,0.990454,0.009546,0
220,1.0,-0.338483,0.279454,0.0,0.0,-0.816612,0.0,0.0,1.679751,0.0,-0.744797,0.0,0.0,0.0,-0.115882,1.056836,0,0.967410,0.032590,0
478,1.0,-2.024845,-0.853102,0.0,0.0,0.253860,0.0,0.0,-1.021386,0.0,0.406273,0.0,1.0,0.0,0.963654,0.040012,0,0.998154,0.001846,0


In [14]:
def print_metrics(y_true, y_pred, tag):
    if isinstance(y_true, pd.DataFrame) or isinstance(y_true, pd.Series):
        y_true = y_true.values
    if isinstance(y_pred, pd.DataFrame) or isinstance(y_pred, pd.Series):
        y_pred = y_pred.values
    if y_true.ndim>1:
        y_true=y_true.ravel()
    if y_pred.ndim>1:
        y_pred=y_pred.ravel()
    val_acc = accuracy_score(y_true, y_pred)
    val_f1 = f1_score(y_true, y_pred)
    print(f"{tag} Acc: {val_acc} | {tag} F1: {val_f1}")

In [15]:
print_metrics(X_test['target'], pred_df["prediction"], tag="Holdout")

Holdout Acc: 0.96 | Holdout F1: 0.7894736842105263


In [16]:
X_val = pd.read_csv('X_val.csv', index_col = 'Unnamed: 0')
y_val = pd.read_csv('y_val.csv', index_col = 'Unnamed: 0')

In [17]:
X_val['target'] = y_val

In [33]:
pred_df = tabular_model.predict(X_val)

In [34]:
print_metrics(X_val['target'], pred_df["prediction"], tag="Holdout")

Holdout Acc: 0.868421052631579 | Holdout F1: 0.8571428571428571


In [20]:
data_config = DataConfig(
    target=['target'],
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform = "quantile_normal"
)

trainer_config = TrainerConfig(
    auto_lr_find = False,
    batch_size = 16,
    max_epochs = 10,
    gpus = 0,
    gradient_clip_val = 10,
    load_best = True
)

model_config = NodeConfig(
    task = "classification",
    num_layers = 3,
    num_trees = 1024,
    learning_rate = 0.1,
    embed_categorical = False,
    metrics = ["accuracy"],
)

tabular_model2 = TabularModel(
    data_config = data_config,
    model_config = model_config,
    optimizer_config = OptimizerConfig(),
    trainer_config = trainer_config,
)

In [21]:
tabular_model2.fit(train = X_train, test = X_test)

C:\Users\olyav\AppData\Roaming\Python\Python38\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\olyav\AppData\Roaming\Python\Python38\site-packages\sklearn\preprocessing\_data.py:2367: UserWarning: n_quantiles (1000) is greater than the total number of samples (640). n_quantiles is set to n_samples.
  warnings.warn("n_quantiles (%s) is greater than the total number "
C:\Users\olyav\AppData\Roaming\Python\Python38\site-packages\pytorch_tabular\models\node\odst.py:152: UserWarning: Data-aware initialization is performed on less than 1000 data points. This may cause instability.To avoid potential problems, run this model on a data batch with at least 1000 data samples.You can do so manually before training. Use with torch.no_grad() for memory efficiency.
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\pytorch_lightning\uti

In [23]:
pred_df = tabular_model2.predict(X_test)
pred_df.head()

,Килип1,ИндНор1,Creatin,Тромболит3,Килип4,ФУ1,ГБ6,АКШ,Возраст,Килип3,Рла1,Килип2,Кл.вар.ИМ1,Хр.оч.инф2,Pсрлао(Адср)1,ПИ,target,0_probability,1_probability,prediction
268,0.0,2.290910,-0.003685,0.0,0.0,-1.030706,0.0,0.0,0.676472,1.0,-0.785664,0.0,0.0,0.0,-2.429174,-0.289795,0,0.671170,0.328830,0
555,0.0,-0.027838,-0.513335,0.0,0.0,1.859569,0.0,0.0,-0.403983,0.0,-0.239671,1.0,0.0,0.0,0.655215,0.040012,0,0.996890,0.003110,0
568,1.0,-0.987511,-1.249497,0.0,0.0,1.591951,0.0,0.0,-0.635509,0.0,-0.938982,0.0,1.0,1.0,-0.578541,0.040012,0,0.995983,0.004017,0
220,1.0,-0.338483,0.279454,0.0,0.0,-0.816612,0.0,0.0,1.679751,0.0,-0.744797,0.0,0.0,0.0,-0.115882,1.056836,0,0.977845,0.022155,0
478,1.0,-2.024845,-0.853102,0.0,0.0,0.253860,0.0,0.0,-1.021386,0.0,0.406273,0.0,1.0,0.0,0.963654,0.040012,0,0.998733,0.001267,0


In [24]:
print_metrics(X_test['target'], pred_df["prediction"], tag="Holdout")

Holdout Acc: 0.965 | Holdout F1: 0.8108108108108109


In [30]:
pred_df = tabular_model2.predict(X_val)
print_metrics(X_val['target'], pred_df["prediction"], tag="Holdout")


Holdout Acc: 0.8421052631578947 | Holdout F1: 0.8235294117647058
